                        Анализ сервиса для чтения книг по подписке

**Цель:** определить наиболее популярные издательства и авторов для выпуска нового издания

##### Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


##### Книги, выпущенные после 01.01.2000

In [6]:
query = '''SELECT count(book_id)
FROM books 
where publication_date > '2000-01-01'


'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,819


После 1 января 2000 года было выпущено в общей сумме 819 книг

##### Обзоры и средняя оценка

In [7]:
query = '''

with
a as (SELECT book_id, count(text) as count_reviews
FROM reviews
GROUP BY book_id),

a1 as (SELECT book_id,
                               ROUND(AVG(rating),2) as avg_rating
                        FROM ratings
                        GROUP BY book_id)
                        
SELECT b.title,
       a.count_reviews, a1.avg_rating
           FROM books as b
           LEFT JOIN a as a ON b.book_id = a.book_id
           LEFT JOIN a1 as a1 ON b.book_id = a1.book_id

order by avg_rating desc, count_reviews desc
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count_reviews,avg_rating
0,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
1,A Dirty Job (Grim Reaper #1),4.0,5.00
2,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,Moneyball: The Art of Winning an Unfair Game,3.0,5.00
4,The Woman in Black,2.0,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,Drowning Ruth,3.0,2.00
997,Junky,2.0,2.00
998,His Excellency: George Washington,2.0,2.00


In [8]:
#кол-во обзоров в изначальной таблице
query = '''

select count(text)
from reviews

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


In [9]:
query = '''

with a as (with
a as (SELECT book_id, count(text) as count_reviews
FROM reviews
GROUP BY book_id),

a1 as (SELECT book_id,
                               ROUND(AVG(rating),2) as avg_rating
                        FROM ratings
                        GROUP BY book_id)
                        
SELECT b.title,
       a.count_reviews, a1.avg_rating
           FROM books as b
           LEFT JOIN a as a ON b.book_id = a.book_id
           LEFT JOIN a1 as a1 ON b.book_id = a1.book_id)
           
select sum(count_reviews)
from a

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,sum
0,2793.0


Таблица содержит информацию о кол-ве отзывов и среднем рейтинге для каждой книги. 

Таким образом, наивысший рейтинг и имеет произведение с названием "The Cat in the Hat and Other Dr. Seuss Favorites", но не имеет отзывов

##### Самое крупное издательство, выпустившее книги толще 50 стр

In [10]:
query = '''

SELECT distinct(publisher), count(distinct(b.book_id))
        
FROM books as b
left join ratings as rate on rate.book_id = b.book_id
left join publishers as publ on publ.publisher_id = b.publisher_id
where b.num_pages > 50
group by distinct(publisher)
order by(count(distinct(b.book_id))) desc
limit 1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


Penguin Books является издательством, которое выпустило максимальное кол-во  уникальных книг толще 50 стр - а именно 42 книги

##### Автор с самой высокой средней оценкой книг 

In [11]:
#будем учитывать только книги с 50 и более оценками
query = '''

with
a1 as (SELECT distinct(a.author), avg(rating) as avg_rating, count(rate.rating_id) as count_reviews
        
FROM books as b
left join ratings as rate on rate.book_id = b.book_id
left join authors as a on a.author_id = b.author_id

group by distinct(a.author)
order by count_reviews desc)

select a1.author
from a1
where count_reviews >=50
limit 1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author
0,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré определена как автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками).

##### Среднее кол-во обзоров от пользователей

In [12]:
#в учет будем брать только тех, кто поставил более 48 оценок
query = '''

WITH a1 AS (SELECT username,
                                        COUNT(rating_id) AS ratings_count
                                 FROM ratings
                                 GROUP BY username
                                 HAVING COUNT(rating_id) > 48),
           a2 AS (SELECT username,
                                   COUNT(review_id) AS reviews_count
                            FROM reviews
                            WHERE username IN (SELECT username FROM a1)
                            GROUP BY username)
SELECT ROUND(AVG(reviews_count),2) AS average_reviews
FROM a2
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,average_reviews
0,24.0


Получается, что среднее кол-во обзоров от пользователей, кто оставил более 48 оценок, равняется 24.

**Полученный вывод:**

После 1.01.2000 выпущено 819 книг, наивысший рейтинг и имеет произведение с названием "The Cat in the Hat and Other Dr. Seuss Favorites", но не имеет отзывов. Penguin Books является издательством, которое выпустило максимальное кол-во уникальных книг толще 50 стр - а именно 42 книги. J.K. Rowling/Mary GrandPré определена как автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками). Среднее кол-во обзоров от пользователей, кто оставил более 48 оценок, равняется 24.
    